In [28]:
import json
import pandas as pd

In [ ]:
! git clone https://github.com/amazon-science/mintaka

In [29]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [31]:
#test

with open('mintaka/data/mintaka_test.json') as user_file:
    test = user_file.read()

parsed_test = json.loads(test)

test = pd.DataFrame(columns=['question', 'question_type'])

for i in range(len(parsed_test)):
    test.loc[i, 'question'] = parsed_test[i]['question']
    test.loc[i, 'question_type'] = parsed_test[i]['complexityType']

In [7]:
#train

with open('mintaka/data/mintaka_train.json') as user_file:
    train = user_file.read()

parsed_train = json.loads(train)

train = pd.DataFrame(columns=['question', 'question_type'])

for i in range(len(parsed_train)):
    train.loc[i, 'question'] = parsed_train[i]['question']
    train.loc[i, 'question_type'] = parsed_train[i]['complexityType']

In [33]:
train.question_type.unique()

array(['ordinal', 'count', 'generic', 'superlative', 'difference',
       'intersection', 'comparative', 'multihop', 'yesno'], dtype=object)

In [9]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

/home/jovyan/.mlspace/envs/konstuktor_lysyuk/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [12]:
train.question_type.value_counts()

question_type
generic         3200
ordinal         1600
count           1600
superlative     1600
difference      1600
intersection    1600
comparative     1600
multihop        1600
yesno           1600
Name: count, dtype: int64

In [13]:
train_data = []
for i in range(len(train)):
    if train.loc[i, 'question_type'] == 'generic':
        cur_train = [train.loc[i, 'question'], 0]
    elif train.loc[i, 'question_type'] == 'ordinal':
        cur_train = [train.loc[i, 'question'], 1]
    elif train.loc[i, 'question_type'] == 'intersection':
        cur_train = [train.loc[i, 'question'], 2]
    elif train.loc[i, 'question_type'] == 'superlative':
        cur_train = [train.loc[i, 'question'], 3]
    elif train.loc[i, 'question_type'] == 'yesno':
        cur_train = [train.loc[i, 'question'], 4]
    elif train.loc[i, 'question_type'] == 'comparative':
        cur_train = [train.loc[i, 'question'], 5]
    elif train.loc[i, 'question_type'] == 'multihop':
        cur_train = [train.loc[i, 'question'], 6]
    elif train.loc[i, 'question_type'] == 'difference':
        cur_train = [train.loc[i, 'question'], 7]
    elif train.loc[i, 'question_type'] == 'count':
        cur_train = [train.loc[i, 'question'], 8]
    train_data.append(cur_train)

In [14]:
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

In [15]:
eval_data = []
for i in range(len(test)):
    if test.loc[i, 'question_type'] == 'generic':
        cur_test = [test.loc[i, 'question'], 0]
    elif test.loc[i, 'question_type'] == 'ordinal':
        cur_test = [test.loc[i, 'question'], 1]
    elif test.loc[i, 'question_type'] == 'intersection':
        cur_test = [test.loc[i, 'question'], 2]
    elif test.loc[i, 'question_type'] == 'superlative':
        cur_test = [test.loc[i, 'question'], 3]
    elif test.loc[i, 'question_type'] == 'yesno':
        cur_test = [test.loc[i, 'question'], 4]
    elif test.loc[i, 'question_type'] == 'comparative':
        cur_test = [test.loc[i, 'question'], 5]
    elif test.loc[i, 'question_type'] == 'multihop':
        cur_test = [test.loc[i, 'question'], 6]
    elif test.loc[i, 'question_type'] == 'difference':
        cur_test = [test.loc[i, 'question'], 7]
    elif test.loc[i, 'question_type'] == 'count':
        cur_test = [test.loc[i, 'question'], 8]
    eval_data.append(cur_test)

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

In [16]:
model_args = ClassificationArgs(num_train_epochs=3)

In [17]:
import torch
cuda_available = torch.cuda.is_available()

In [18]:
model = ClassificationModel(
    "bert", "bert-base-uncased", num_labels = 9, args=model_args, cuda_device=0
)

True

In [ ]:
model.train_model(train_df, show_running_loss = False)

In [18]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel("bert", './outputs', use_cuda=True)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [21]:
import numpy as np
for i in range(len(eval_df)):
    eval_df.loc[i, 'pred_label'] = np.argmax(model_outputs[i])

In [22]:
len(eval_df.query("labels == pred_label"))/len(eval_df)

0.92425